In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_decomposition import CCA
import matplotlib.pyplot as plt

# 소수를 e로 표현하지 않도록 하기
pd.options.display.float_format = "{:.5f}".format

In [2]:
lib_2007 = pd.read_csv("final_data/lib_2007.csv", index_col=0)
lib_2008 = pd.read_csv("final_data/lib_2008.csv", index_col=0)
lib_2009 = pd.read_csv("final_data/lib_2009.csv", index_col=0)
lib_2010 = pd.read_csv("final_data/lib_2010.csv", index_col=0)
lib_2011 = pd.read_csv("final_data/lib_2011.csv", index_col=0)
lib_2012 = pd.read_csv("final_data/lib_2012.csv", index_col=0)
lib_2013 = pd.read_csv("final_data/lib_2013.csv", index_col=0)
lib_2014 = pd.read_csv("final_data/lib_2014.csv", index_col=0)
lib_2015 = pd.read_csv("final_data/lib_2015.csv", index_col=0)
lib_2016 = pd.read_csv("final_data/lib_2016.csv", index_col=0)
lib_2017 = pd.read_csv("final_data/lib_2017.csv", index_col=0)
lib_2018 = pd.read_csv("final_data/lib_2018.csv", index_col=0)
lib_2019 = pd.read_csv("final_data/lib_2019.csv", index_col=0)
lib_2020 = pd.read_csv("final_data/lib_2020.csv", index_col=0)
lib_2021 = pd.read_csv("final_data/lib_2021.csv", index_col=0)

In [3]:
train_lib = pd.concat([lib_2007,lib_2008,lib_2009,lib_2010,lib_2011,lib_2012,lib_2013,lib_2014,lib_2015,lib_2016,lib_2017])
valid_lib = pd.concat([lib_2018, lib_2019])
test_lib = pd.concat([lib_2020,lib_2021])

In [4]:
# 이상치 제거 전 train, test set의 데이터 수 구하기
print(f"이상치 제거 전 train set의 데이터 수 : {len(train_lib)}")
print(f"이상치 제거 전 valid set의 데이터 수 : {len(valid_lib)}")
print(f"이상치 제거 전 test set의 데이터 수 : {len(test_lib)}")

이상치 제거 전 train set의 데이터 수 : 9026
이상치 제거 전 valid set의 데이터 수 : 2222
이상치 제거 전 test set의 데이터 수 : 2370


In [5]:
# 사분위수를 활용하여 비율 충족도의 이상치 행 제거하기
def del_outlier(df, col):
    q1 = df[col].quantile(0.10)
    q3 = df[col].quantile(0.90)
    iqr = q3 - q1 
    boundary = 1.5 * iqr 

    upper_index = df[df[col] > q3 + boundary].index
    lower_index = df[df[col] < q1 - boundary].index 

    df.drop(upper_index, inplace = True)
    df.drop(lower_index, inplace = True)

    return df

del_outlier(train_lib, "future_acq_budget_settlement")
del_outlier(valid_lib, "future_acq_budget_settlement")
del_outlier(test_lib, "future_acq_budget_settlement") 

# 이상치 제거 후 train, test set의 데이터 수 구하기
print(f"이상치 제거 후 train set의 데이터 수 : {len(train_lib)}")
print(f"이상치 제거 후 valid set의 데이터 수 : {len(valid_lib)}")
print(f"이상치 제거 후 test set의 데이터 수 : {len(test_lib)}")

이상치 제거 후 train set의 데이터 수 : 8184
이상치 제거 후 valid set의 데이터 수 : 2178
이상치 제거 후 test set의 데이터 수 : 2310


In [6]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max Scaler 생성
scaler = MinMaxScaler()

# 모든 변수에 대해 Min-Max Scaling 적용
# 스케일러를 학습 데이터에 대해 fit
scaler.fit(train_lib.drop(columns=["future_acq_budget_settlement"]))

# 학습 데이터에 대해 변환
train_lib_scaled = pd.DataFrame(scaler.transform(train_lib.drop(columns=["future_acq_budget_settlement"])), 
                                columns=train_lib.columns[:-1])

# 타겟 변수에 대해 스케일링 적용
scaler_y = MinMaxScaler()
train_y_scaled = scaler_y.fit_transform(train_lib[['future_acq_budget_settlement']])

# 학습 데이터에 스케일된 타겟 변수 추가
train_lib_scaled['future_acq_budget_settlement_scaled'] = train_y_scaled

# 검증 데이터에 대해 변환
valid_lib_scaled = pd.DataFrame(scaler.transform(valid_lib.drop(columns=["future_acq_budget_settlement"])), 
                                columns=train_lib.columns[:-1])
valid_y_scaled = scaler_y.transform(valid_lib[['future_acq_budget_settlement']])
valid_lib_scaled['future_acq_budget_settlement_scaled'] = valid_y_scaled

# 테스트 데이터에 대해 변환
test_lib_scaled = pd.DataFrame(scaler.transform(test_lib.drop(columns=["future_acq_budget_settlement"])), 
                               columns=train_lib.columns[:-1])
test_y_scaled = scaler_y.transform(test_lib[['future_acq_budget_settlement']])
test_lib_scaled['future_acq_budget_settlement_scaled'] = test_y_scaled

# X_train, Y_train 구성하기 (스케일된 타겟 사용)
X_train = train_lib_scaled.drop(columns=["future_acq_budget_settlement_scaled"])
Y_train = train_lib_scaled["future_acq_budget_settlement_scaled"]

# X_valid, Y_valid 구성하기 (스케일된 타겟 사용)
X_valid = valid_lib_scaled.drop(columns=["future_acq_budget_settlement_scaled"])
Y_valid = valid_lib_scaled["future_acq_budget_settlement_scaled"]

# X_test, Y_test 구성하기 (스케일된 타겟 사용)
X_test = test_lib_scaled.drop(columns=["future_acq_budget_settlement_scaled"])
Y_test = test_lib_scaled["future_acq_budget_settlement_scaled"]


In [7]:
feature_names = X_train.columns

In [8]:
import pandas as pd
from sklearn.cross_decomposition import CCA
from sklearn.metrics import r2_score

# 결과 저장을 위한 리스트
correlations = []
r2_scores = []

# 각 입력 특징에 대해 CCA 적용
for i in range(X_train.shape[1]):
    # 입력 특징 개별 추출 (DataFrame의 경우 iloc 사용)
    X_single = X_train.iloc[:, i].values.reshape(-1, 1)
    
    # CCA 모델 초기화
    cca = CCA(n_components=1)
    
    # CCA 적용
    cca.fit(X_single, Y_train)
    X_c, Y_c = cca.transform(X_single, Y_train)
    
    # 상관 계수
    corr = np.corrcoef(X_c.T, Y_c.T)[0, 1]
    correlations.append(corr)
    
    # R^2 값
    r2 = r2_score(X_c, Y_c)
    r2_scores.append(r2)

# 결과를 데이터프레임으로 정리
results_df = pd.DataFrame({
    'Feature': X_train.columns,  # 특성 이름
    'Correlation': correlations,
    'R^2': r2_scores
})

# 결과 출력
print(results_df)




                  Feature  Correlation      R^2
0               dom_books      0.64004  0.28008
1               for_books      0.33375 -0.33249
2               local_mat      0.18910 -0.62181
3             nonbook_mat      0.46552 -0.06895
4                elec_mat      0.06683 -0.86634
5                 serials      0.30519 -0.38962
6              ann_growth      0.15395 -0.69210
7             ann_weeding      0.25069 -0.49863
8           lib_site_area      0.11731 -0.76539
9          lib_build_area      0.35730 -0.28539
10            total_seats      0.60690  0.21380
11             user_comps      0.61875  0.23750
12      self_srv_machines      0.38036 -0.23928
13              full_time      0.64362  0.28724
14              part_time      0.47188 -0.05624
15          support_staff      0.20651 -0.58699
16           total_budget      0.62198  0.24396
17             acq_budget      0.74814  0.49627
18              open_days      0.02571 -0.94858
19         avg_week_hours      0.04845 -